# Assignment 3: Build a seq2seq model for machine translation.

### Name: Atul Gupta

### Task: Change LSTM model to Bidirectional LSTM Model and Translate English to Spanish

### Due Date: Wednesday, April 17th, 11:59PM

## 0. You will do the following:

1. Read and run the code. Please make sure you have installed keras or tensorflow.Running the script on colab will speed up the training process and also prevent package loading issue.
2. Complete the code in Section 1.1, you may fill in your data directory.
3. Directly modify the code in Section 3. Change the current LSTM layer to a Bidirectional LSTM Model.
4. Training your model and translate English to Spanish in Section 4.2. You could try translating other languages.
5. Complete the code in Section 5.

### Hint:

To implement ```Bi-LSTM```, you will need the following code to build the encoder **in Section 3**. Do NOT use Bi-LSTM for the decoder. But there are other codes **you need to modify** to make it work.

In [1]:
# from keras.layers import Bidirectional, Concatenate

# encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True,
#                                   dropout=0.5, name='encoder_lstm'))
# _, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])

## 1. Data preparation (10 points)

1. Download spanish-english data from http://www.manythings.org/anki/
2. You may try to use other languages.
3. Unzip the .ZIP file.
4. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".
5. Fill in your data directory in section 1.1.

### 1.1. Load and clean text


In [2]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [3]:
# e.g., filename = 'Data/deu.txt'
filename = "/Users/atul/Library/CloudStorage/OneDrive-Personal/Stevens Institute of Technology/Courses/CS 583/HW 3/Data/spa.txt"

# e.g., n_train = 20000
n_train = 20000

In [4]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [5]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[we are here] => [estamos aqui]
[we ate eggs] => [hemos comido huevos]
[we ate eggs] => [comimos huevos]
[we broke up] => [nos separamos]
[we broke up] => [lo dejamos]
[we broke up] => [rompimos]
[we can help] => [podemos ayudar]
[we can help] => [nosotros podemos ayudar]
[we can meet] => [podemos encontrarnos]
[we can meet] => [podemos vernos]


In [6]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (20000,)
Length of target_texts: (20000,)


In [7]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 48


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length,
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length,
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

2024-04-17 12:09:46.754502: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 48)
shape of target_token_index: 29


In [9]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices.

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [10]:
target_texts[100]

'\tentendiste\n'

In [11]:
decoder_input_seq[100, :]

array([ 6,  2,  9,  8,  2,  9, 15, 11,  5,  8,  2,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [12]:
from tensorflow.keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq,
                                    max_decoder_seq_length,
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 48, 30)


## 3. Build the networks (for training) (20 points)

- In this section, we have already implemented the LSTM model for you. You can run the code and see what the code is doing.  

- **You need to change the existing LSTM model to a Bidirectional LSTM model. Just modify the network structure and do not change the training cell in section 3.4.**

- Build encoder, decoder, and connect the two modules to get "model".

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.



### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return:

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [13]:
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Concatenate
from tensorflow.keras.models import Model

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens),
                       name='encoder_inputs')

# # set the LSTM layer
# encoder_lstm = LSTM(latent_dim, return_state=True,
#                     dropout=0.5, name='encoder_lstm')
# _, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                  dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs,
                      outputs=[state_h, state_c],
                      name='encoder')

2024-04-17 12:09:51.086152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Print a summary and save the encoder network structure to "./encoder.pdf"

In [14]:
from IPython.display import SVG
from keras.utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None, 28)]   0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 512),        583680      ['encoder_inputs[0][0]']         
                                 (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$
    
    -- The initial conveyor belt $c_t$

- Return:

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [15]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

latent_dim = 512

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True,
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x,
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

Print a summary and save the encoder network structure to "./decoder.pdf"

In [16]:
from IPython.display import SVG
from keras.utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 decoder_input_h (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 decoder_input_c (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 512),  1112064     ['decoder_input_x[0][0]',        
                                 (None, 512),                     'decoder_input_h[0][0]',  

### 3.3. Connect the encoder and decoder

In [17]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x],
              outputs=decoder_pred,
              name='model_training')

In [18]:
from IPython.display import SVG
from keras.utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_x (InputLayer)   [(None, None, 28)]   0           []                               
                                                                                                  
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 encoder (Functional)           [(None, 512),        583680      ['encoder_input_x[0][0]']        
                                 (None, 512)]                                                     
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 512),  1112064     ['decoder_input_x[0]

### 3.4. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [19]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 48, 30)
shape of decoder_target_data(20000, 48, 30)


In [20]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [21]:
from tensorflow import keras

def model_builder(hp):

    model = Model(inputs=[encoder_input_x, decoder_input_x], 
                outputs=decoder_pred, 
                name='model_training')
  
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd', 'adagrad', 'adamax'])

    optimizer = {
        'adam':keras.optimizers.Adam(learning_rate=hp_learning_rate),
        'rmsprop': keras.optimizers.RMSprop(learning_rate=hp_learning_rate),
        'sgd': keras.optimizers.SGD(learning_rate=hp_learning_rate),
        'adagrad': keras.optimizers.Adagrad(learning_rate=hp_learning_rate),
        'adamax': keras.optimizers.Adamax(learning_rate=hp_learning_rate)
    }[hp_optimizer]

    model.compile(optimizer=optimizer, loss='categorical_crossentropy')

    return model

In [22]:
import keras_tuner as kt
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=50,
                     factor=3,
                     directory='model_dir',
                     project_name='test5')

stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

In [23]:
from tensorflow import keras
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy')

model.save_weights('model_weights.h5')

In [24]:
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
history = model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50
250/250 [==============================] - 86s 334ms/step - loss: 1.0289 - val_loss: 0.9562
Epoch 2/50
250/250 [==============================] - 96s 385ms/step - loss: 0.8287 - val_loss: 0.8623
Epoch 3/50
250/250 [==============================] - 97s 389ms/step - loss: 0.7921 - val_loss: 0.8157
Epoch 4/50
250/250 [==============================] - 99s 397ms/step - loss: 0.7647 - val_loss: 0.7822
Epoch 5/50
250/250 [==============================] - 97s 388ms/step - loss: 0.7400 - val_loss: 0.7510
Epoch 6/50
250/250 [==============================] - 98s 392ms/step - loss: 0.7202 - val_loss: 0.7172
Epoch 7/50
250/250 [==============================] - 96s 384ms/step - loss: 0.7011 - val_loss: 0.6948
Epoch 8/50
250/250 [==============================] - 107s 427ms/step - loss: 0.6822 - val_loss: 0.6740
Epoch 9/50
250/250 [==============================] - 101s 402ms/step - loss: 0.6673 - val_loss: 0.6507
Epoch 10/50
250/250 [==============================] - 101s 404ms/step 

In [25]:
val_loss_per_epoch = history.history['val_loss']
best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1

print('Min val loss: %d' % (min(val_loss_per_epoch)))
print('Best epoch: %d' % (best_epoch,))

Min val loss: 0
Best epoch: 49


## 4. Make predictions

- In this section, you need to complete section 4.2 to translate English to the target language.


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [26]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [27]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])

        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [28]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


1/1 [==============================] - 0s 22ms/step
-
English:        come see me
Spanish (true):  venid a verme
Spanish (pred):  ven a casa
1/1 [==============================] - 0s 19ms/step
-
English:        come see me
Spanish (true):  vengan a verme
Spanish (pred):  ven a casa
1/1 [==============================] - 0s 20ms/step
-
English:        come see me
Spanish (true):  venga a verme
Spanish (pred):  ven a casa
1/1 [==============================] - 0s 21ms/step
-
English:        comfort tom
Spanish (true):  consuela a tom
Spanish (pred):  ven a con tom
1/1 [==============================] - 0s 20ms/step
-
English:        contact tom
Spanish (true):  ponte en contacto con tom
Spanish (pred):  contenta a tom
1/1 [==============================] - 0s 19ms/step
-
English:        cook for me
Spanish (true):  cociname
Spanish (pred):  cuenta a casa masm
1/1 [==============================] - 0s 19ms/step
-
English:        cook for me
Spanish (true):  cocina para mi
Spanish (pred): 

1/1 [==============================] - 0s 19ms/step
-
English:        count on it
Spanish (true):  cuente con eso
Spanish (pred):  cuenta con eso
1/1 [==============================] - 0s 19ms/step
-
English:        count on me
Spanish (true):  cuenta conmigo
Spanish (pred):  cuenta conmigo
1/1 [==============================] - 0s 19ms/step
-
English:        did tom die
Spanish (true):  ha muerto tom
Spanish (pred):  lo hijo tom
1/1 [==============================] - 0s 19ms/step
-
English:        did tom run
Spanish (true):  corrio tom
Spanish (pred):  lo piobo tom
1/1 [==============================] - 0s 19ms/step
-
English:        did tom try
Spanish (true):  lo ha probado tom
Spanish (pred):  lo habo tom
1/1 [==============================] - 0s 19ms/step
-
English:        did tom try
Spanish (true):  al final lo probo tom o no
Spanish (pred):  lo habo tom
1/1 [==============================] - 0s 19ms/step
-
English:        did tom try
Spanish (true):  lo probo tom
Spanish (pred

1/1 [==============================] - 0s 21ms/step
-
English:        dig quickly
Spanish (true):  cavad rapido
Spanish (pred):  habla frinces


### 4.2. Translate an English sentence to the target language （20 points）

1. Tokenization
2. One-hot encode
3. Translate

In [29]:
input_sentence = 'I love you'

input_sequence = [[input_token_index[char] for char in input_sentence.lower()]]
input_sequence_padded = pad_sequences(input_sequence, maxlen=max_encoder_seq_length, padding='post')

input_x = onehot_encode(input_sequence_padded, max_encoder_seq_length, num_encoder_tokens)

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + translated_sentence)

1/1 [==============================] - 0s 19ms/step
source sentence is: I love you
translated sentence is: te amaba



# 5. Evaluate the translation using BLEU score

- We have already translated from English to target language, but how can we evaluate the performance of our model quantitatively?

- In this section, you need to re-train the model we built in section 3 and then evaluate the bleu score on testing dataset.

Reference:

https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

https://en.wikipedia.org/wiki/BLEU

#### Hint:

- Randomly partition the dataset to training, validation, and test.

- Evaluate the BLEU score using the test set. Report the average.

- You may use packages to calculate bleu score, e.g., sentence_bleu() from nltk package.

### 5.1. Partition the dataset to training, validation, and test. Build new token index. (10 points)

1. You may try to load more data/lines from text file.
2. Convert text to sequences and build token index using training data.
3. One-hot encode your training and validation text sequences.

In [30]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)

In [31]:
total_size = 100000
rand_indices = numpy.random.permutation(total_size)
train_indices = rand_indices[0:50000]
val_indices = rand_indices[50000:70000]
test_indices = rand_indices[70000:100000]

clean_pairs_train = clean_pairs[train_indices, :]
clean_pairs_val = clean_pairs[val_indices, :]
clean_pairs_test = clean_pairs[test_indices, :]

In [32]:
# Training Data
input_texts_train = clean_pairs_train[:, 0]
target_texts_train = ['\t' + text + '\n' for text in clean_pairs_train[:, 1]]

# Validation Data
input_texts_val = clean_pairs_val[:, 0]
target_texts_val = ['\t' + text + '\n' for text in clean_pairs_val[:, 1]]

# Testing Set
input_texts_test = clean_pairs_test[:, 0]
target_texts_test = ['\t' + text + '\n' for text in clean_pairs_test[:, 1]]

In [33]:
max_encoder_seq_length = max(len(line) for line in input_texts_train)
max_decoder_seq_length = max(len(line) for line in target_texts_train)

In [34]:
encoder_input_seq_train, input_token_index_train = text2sequences(max_encoder_seq_length, 
                                                      input_texts_train)
decoder_input_seq_train, target_token_index_train = text2sequences(max_decoder_seq_length, 
                                                       target_texts_train)

print('shape of encoder_input_seq_train: ' + str(encoder_input_seq_train.shape))
print('shape of input_token_index_train: ' + str(len(input_token_index_train)))
print('shape of decoder_input_seq_train: ' + str(decoder_input_seq_train.shape))
print('shape of target_token_index_train: ' + str(len(target_token_index_train)))

shape of encoder_input_seq_train: (50000, 35)
shape of input_token_index_train: 27
shape of decoder_input_seq_train: (50000, 87)
shape of target_token_index_train: 29


In [35]:
encoder_input_seq_val, input_token_index_val = text2sequences(max_encoder_seq_length, 
                                                      input_texts_val)
decoder_input_seq_val, target_token_index_val = text2sequences(max_decoder_seq_length, 
                                                       target_texts_val)

print('shape of encoder_input_seq_val: ' + str(encoder_input_seq_val.shape))
print('shape of input_token_index_val: ' + str(len(input_token_index_val)))
print('shape of decoder_input_seq_val: ' + str(decoder_input_seq_val.shape))
print('shape of target_token_index_val: ' + str(len(target_token_index_val)))

shape of encoder_input_seq_val: (20000, 35)
shape of input_token_index_val: 27
shape of decoder_input_seq_val: (20000, 87)
shape of target_token_index_val: 29


In [36]:
num_encoder_tokens = len(input_token_index_train) + 1
num_decoder_tokens = len(target_token_index_train) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


In [37]:
encoder_input_data_train = onehot_encode(encoder_input_seq_train, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data_train = onehot_encode(decoder_input_seq_train, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq_train = numpy.zeros(decoder_input_seq_train.shape)
decoder_target_seq_train[:, 0:-1] = decoder_input_seq_train[:, 1:]
decoder_target_data_train = onehot_encode(decoder_target_seq_train, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data_train.shape)
print(decoder_input_data_train.shape)

(50000, 35, 28)
(50000, 87, 30)


In [38]:
encoder_input_data_val = onehot_encode(encoder_input_seq_val, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data_val = onehot_encode(decoder_input_seq_val, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq_val = numpy.zeros(decoder_input_seq_val.shape)
decoder_target_seq_val[:, 0:-1] = decoder_input_seq_val[:, 1:]
decoder_target_data_val = onehot_encode(decoder_target_seq_val, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data_val.shape)
print(decoder_input_data_val.shape)

(20000, 35, 28)
(20000, 87, 30)


### 5.2 Retrain your previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score. (25 points)

1. Use the model structure in section 3 to train a new model with new training and validation datasets.
2. Based on validation BLEU score or loss to tune parameters.

In [39]:
from tensorflow import keras

model.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.01),
              loss='categorical_crossentropy')

stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

model.fit([encoder_input_data_train, decoder_input_data_train],
               decoder_target_data_train, epochs=50,
               validation_data=([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val),
               callbacks=[stop_early])

# Saving retrained model
model.save('seq2seq_retrained.h5')
model.save_weights('model_weights_retrained.h5')

Epoch 1/50
1563/1563 [==============================] - 767s 489ms/step - loss: 0.6829 - val_loss: 0.4948
Epoch 2/50
1563/1563 [==============================] - 755s 483ms/step - loss: 0.5984 - val_loss: 0.4276
Epoch 3/50
1563/1563 [==============================] - 809s 517ms/step - loss: 0.5617 - val_loss: 0.3944
Epoch 4/50
1563/1563 [==============================] - 813s 520ms/step - loss: 0.5367 - val_loss: 0.3682
Epoch 5/50
1563/1563 [==============================] - 818s 523ms/step - loss: 0.5179 - val_loss: 0.3567
Epoch 6/50
1563/1563 [==============================] - 785s 502ms/step - loss: 0.5042 - val_loss: 0.3417
Epoch 7/50
1563/1563 [==============================] - 763s 488ms/step - loss: 0.4917 - val_loss: 0.3332
Epoch 8/50
1563/1563 [==============================] - 760s 486ms/step - loss: 0.4845 - val_loss: 0.3249
Epoch 9/50
1563/1563 [==============================] - 760s 486ms/step - loss: 0.4742 - val_loss: 0.3205
Epoch 10/50
1563/1563 [=======================

### 5.3 Evaluate the BLEU score using the test set. (15 points)

1. Use trained model above to calculate the BLEU score with testing dataset.
2. A reasonable value should be 0.1-0.3. The higher, the better.

In [40]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smoothing_function = SmoothingFunction().method2

blue_scores = []

encoder_input_seq_test, input_token_index_test = text2sequences(max_encoder_seq_length, 
                                                      input_texts_test)

encoder_input_data_test = onehot_encode(encoder_input_seq_test, max_encoder_seq_length, num_encoder_tokens)

reverse_input_char_index = dict((i, char) for char, i in input_token_index_train.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index_train.items())

encoder_input_seq_test, input_token_index_test = text2sequences(max_encoder_seq_length, 
                                                      input_texts_test)
decoder_input_seq_test, target_token_index_test = text2sequences(max_decoder_seq_length, 
                                                       target_texts_test)

In [41]:
print('shape of encoder_input_seq_test: ' + str(encoder_input_seq_test.shape))
print('shape of input_token_index_test: ' + str(len(input_token_index_test)))
print('shape of decoder_input_seq_test: ' + str(decoder_input_seq_test.shape))
print('shape of target_token_index_test: ' + str(len(target_token_index_test)))

shape of encoder_input_seq_test: (30000, 35)
shape of input_token_index_test: 27
shape of decoder_input_seq_test: (30000, 87)
shape of target_token_index_test: 29


In [42]:
encoder_input_data_test = onehot_encode(encoder_input_seq_test, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data_test = onehot_encode(decoder_input_seq_test, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq_test = numpy.zeros(decoder_input_seq_test.shape)
decoder_target_seq_test[:, 0:-1] = decoder_input_seq_test[:, 1:]
decoder_target_data_test = onehot_encode(decoder_target_seq_test, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

In [43]:
print(encoder_input_data_test.shape)
print(decoder_input_data_test.shape)

(30000, 35, 28)
(30000, 87, 30)


In [44]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index_test.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index_test.items())

In [45]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose='none')

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value,
                                                    verbose='none')

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [46]:
bleu_scores = []
random_sample = numpy.random.randint(low=0, high=30000, size=100)

for i, seq_index in enumerate(random_sample):
    input_seq = encoder_input_data_test[seq_index: seq_index + 1]
    target = target_texts_test[seq_index]
    try:
        decoded_sentence = decode_sequence(input_seq)
    except KeyError as err:
        continue

    score = sentence_bleu(target, decoded_sentence, smoothing_function=smoothing_function)
    print(i, '->', score)
    bleu_scores.append(score)

0 -> 0.12300686288463772
1 -> 0.060820553106281094
2 -> 0.11044795567078944
3 -> 0.0894256643143163
4 -> 0.13761232669480913
5 -> 0.06730900646889454
6 -> 0.05648892151960283
7 -> 0.06664199814563841
8 -> 0.05599379119409926
9 -> 0.059239446585176526
10 -> 0.05478891918239913
11 -> 0.09224978011409782
12 -> 0.057279598784164755
13 -> 0.06494357672103247
14 -> 0.07837993901367481
15 -> 0.060820553106281094
16 -> 0.07046533377745744
17 -> 0.09500653305583746
18 -> 0.04762330565538855
19 -> 0.059579874656163496
20 -> 0.0588083967791575
21 -> 0.037796447300922714
22 -> 0.06279501637208412
23 -> 0.13309426770177107
24 -> 0.058898054573110994
25 -> 0.11678449443205002
26 -> 0.1280220256953781
27 -> 0.0721645129273987
28 -> 0.09469491246884826
29 -> 0.06941254212572458
30 -> 0.04718595618828885
31 -> 0.07997892972095699
32 -> 0.059579874656163496
33 -> 0.09754226655135786
34 -> 0.1344791119656338
35 -> 0.10682175159905853
36 -> 0.058297837453414166
37 -> 0.09469491246884826
38 -> 0.0533750765

In [47]:
bleu_sum = sum(bleu_scores)
total = len(bleu_scores)
print(bleu_sum/total)

0.07601462069041984
